In [1]:
# Installs geemap package
import subprocess
import os
import pandas as pd
import geopandas as gpd
import json #do I need this if I'm using geopandas?
import rasterio as rst
import ipywidgets as widgets
from IPython.display import display

import geemap as geemap
#import geemap.foliumap as geemap
import ee

In [2]:
#ee.Authenticate()
ee.Initialize()

input_year = widgets.Select(
    options=[' ',2018, 2019, 2020, 2021],
    value=' ',
    description="Land clearing after:",
    disabled=False
)

vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
tree_vis_params = {'min':0, 'max':1, 'palette':['white','green']}
change_vis_params = {'min':0, 'max':1, 'palette':['white','red']}


SHOW_TREES = False
SHOW_S2 = False
SHOW_DIFF = True

geom_gpd = gpd.read_file('data/marysville-fire-forest-loss-test-area.geojson')
geom = geemap.geopandas_to_ee(geom_gpd)

#widget to control where map output goes
map_output = widgets.Output()

m = geemap.Map(height="800px", width="1600px")
m.add_basemap('HYBRID')
m.addLayer(geom, {})


In [3]:
def masks2clouds(image):
    """
    Make a sentinel-2 true colour image of the most recent calendar year as a visual aid.
    maskS2clouds uses metadata to remove cloudy pixels before creating a composite image based on the filter dates.
    If you want to mess with this section of code, check with Jenna first.
    Make sure to use the Harmonized Sentinel-2 collection, not the raw surface reflectance collection.
    If you remove the divide/10000 you need to also update the vis_params further down.
    """
    cloud_pixel_threshold = 25
    s2_start_date = '2022-01-01'
    s2_end_date = '2022-12-31'

    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])



In [4]:
def dynamicworld_operation(new_year):
    """
    The 'label' band is used because its the index of the band with the highest estimated probability. 
    So, if, based on all the pixels in the collection the most likely class is 'trees' it will be assigned a value of 1 in the labels band.

    Set to 01-01-2018 because thats the first full year of sentinel-2 imagery.
    its the earliest year we should offer in the POC
    """
    start_date = "2018-01-01"
    before_date = str(input_year.value) + "-12-31"
    after_date = str(input_year.value+1) + "-01-01"

    geom = geemap.geopandas_to_ee(geom_gpd)

    """
    TODO: change end_date to be based dynamically on current date.
    """
    end_date = "2023-10-01"
    
    dynamicworld_collection_before_median = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate(start_date, before_date).select('label').median()
    dynamicworld_collection_after_median = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate(after_date, end_date).select('label').median()
    dynamicworld_trees_before_median = dynamicworld_collection_before_median.eq(1).clip(geom)
    dynamicworld_trees_after_median = dynamicworld_collection_after_median.eq(1).clip(geom)

    dynamicworld_tree_difference_before_after_median = dynamicworld_collection_before_median.eq(1).And(dynamicworld_collection_after_median.neq(1)).clip(geom)

    m.centerObject(geom)
    m.addLayer(dynamicworld_tree_difference_before_after_median.updateMask(dynamicworld_tree_difference_before_after_median), change_vis_params, 'tree loss since '+str(input_year.value), shown=SHOW_DIFF)
    
    #m.addLayer(dynamicworld_trees_before_median.updateMask(dynamicworld_trees_before_median), tree_vis_params, 'Pre-'+str(input_year.value)+' trees', shown=SHOW_TREES)
    #m.addLayer(dynamicworld_trees_after_median.updateMask(dynamicworld_trees_after_median), tree_vis_params, 'Post-'+str(input_year.value)+' trees', shown=SHOW_TREES)
    #m.addLayer(composite, vis_params, 'Sentinel2 RGB', shown=SHOW_S2)
    #make s2 composite
    #collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(s2_start_date, s2_end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_pixel_threshold)).map(maskS2clouds)
    #composite = collection.median().clip(geom)
    #m.addLayer(composite, vis_params, 'Sentinel2 RGB')

In [5]:
def on_year_change(change):
    with map_output:
        new_year = change['new']
        dynamicworld_operation(new_year)


In [6]:
input_year.observe(on_year_change, names='value')
display(input_year)


Select(description='Land clearing after:', options=(' ', 2018, 2019, 2020, 2021), value=' ')

In [7]:
display(map_output)
with map_output:
    display(m)

Output()